In [35]:
import pandas as pd
from scipy.stats import norm

In [49]:
df = pd.read_excel('Base de dato Apuestas Deportivas.xlsx')

In [55]:
# Asumiendo que 'df' es tu DataFrame original con los datos de los partidos

# Ampliar el diccionario para almacenar las desviaciones estándar
resultados = {
    'Equipo': [],
    'Goles L': [],
    'Goles V': [],
    'Goles L EC': [],
    'Goles V EC': [],
    'Tiros L': [],
    'Tiros V': [],
    'Tiros L EC': [],
    'Tiros V EC': [],
    'Tiros OT L': [],
    'Tiros OT V': [],
    'Tiros OT L EC': [],
    'Tiros OT V EC': [],
    'Tarjetas L': [],
    'Tarjetas V': [],
    'Tarjetas L EC': [],
    'Tarjetas V EC': [],
    'Corners L': [],
    'Corners V': [],
    'Corners L EC': [],
    'Corners V EC': [],
    # Desviaciones estándar
    'Goles L SD': [],
    'Goles V SD': [],
    'Goles L EC SD': [],
    'Goles V EC SD': [],
    'Tiros L SD': [],
    'Tiros V SD': [],
    'Tiros L EC SD': [],
    'Tiros V EC SD': [],
    'Tiros OT L SD': [],
    'Tiros OT V SD': [],
    'Tiros OT L EC SD': [],
    'Tiros OT V EC SD': [],
    'Tarjetas L SD': [],
    'Tarjetas V SD': [],
    'Tarjetas L EC SD': [],
    'Tarjetas V EC SD': [],
    'Corners L SD': [],
    'Corners V SD': [],
    'Corners L EC SD': [],
    'Corners V EC SD': []
}

def agregar_promedios_y_desviaciones(df, equipo):
    # Local
    df_local = df[df['Local'] == equipo].sort_values(by='Fecha', ascending=False).head(10)
    df_visitante = df[df['Visitante'] == equipo].sort_values(by='Fecha', ascending=False).head(10)
    
    # Agregar promedios y desviaciones estándar al diccionario
    for metrica in ['Goles L', 'Goles V', 'Tiros L', 'Tiros V', 'Tiros OT L', 'Tiros OT V', 'Tarjetas L', 'Tarjetas V', 'Corners L', 'Corners V']:
        if "L" in metrica:    
            resultados[metrica].append(df_local[metrica].mean())
            resultados[f"{metrica} EC"].append(df_local[f"{metrica.split(' ')[0]} V"].mean())
            resultados[f"{metrica} SD"].append(df_local[metrica].std())
            resultados[f"{metrica} EC SD"].append(df_local[f"{metrica.split(' ')[0]} V"].std())

        if 'V' in metrica:
            resultados[metrica].append(df_visitante[metrica].mean())
            resultados[f"{metrica} EC"].append(df_visitante[f"{metrica.split(' ')[0]} L"].mean())
            resultados[f"{metrica} SD"].append(df_visitante[metrica].std())
            resultados[f"{metrica} EC SD"].append(df_visitante[f"{metrica.split(' ')[0]} L"].std())

    resultados['Equipo'].append(equipo)

# Calcular promedios y desviaciones estándar para cada equipo
equipos = pd.unique(df[['Local', 'Visitante']].values.ravel('K'))
for equipo in equipos:
    agregar_promedios_y_desviaciones(df, equipo)

# Crear el DataFrame final a partir del diccionario de resultados
df = pd.DataFrame(resultados)


In [66]:
from scipy.stats import poisson

# Re-definir la función para calcular probabilidades usando el DataFrame corregido
def calcular_probabilidades_normal(df, equipo_local, equipo_visitante, categoria, umbrales):
    # Extraer los promedios y desviaciones estándar para el equipo local y visitante
    # Filtrar el DataFrame para obtener datos del equipo local y visitante
    datos_equipo_local = df[df['Equipo'] == equipo_local]
    datos_equipo_visitante = df[df['Equipo'] == equipo_visitante]

    # Asumir que cada DataFrame filtrado (datos_equipo_local y datos_equipo_visitante) tiene exactamente una fila
    promedio_local = datos_equipo_local[f"{categoria} L"].values[0]
    promedio_visitante = datos_equipo_visitante[f"{categoria} V"].values[0]
    desviacion_local = datos_equipo_local[f"{categoria} L SD"].values[0] if f"{categoria} L SD" in datos_equipo_local else 0
    desviacion_visitante = datos_equipo_visitante[f"{categoria} V SD"].values[0] if f"{categoria} V SD" in datos_equipo_visitante else 0

    promedio_combinado = promedio_local + promedio_visitante
    desviacion_combinada = (desviacion_local**2 + desviacion_visitante**2)**0.5

    resultados = {}
    for umbral in umbrales:
        prob_over = 1 - norm.cdf(umbral, loc=promedio_combinado, scale=desviacion_combinada)
        prob_under = norm.cdf(umbral, loc=promedio_combinado, scale=desviacion_combinada)
        resultados[f"Over/Under {umbral}"] = {"Over": prob_over, "Under": prob_under}
    
    return resultados

# Categorías y umbrales para calcular
categorias_y_umbrales = {
    "Goles": [0.5, 1.5, 2.5, 3.5],
    "Corners": [7.5, 8.5, 9.5, 10.5, 11.5],
    "Tiros": [20.5, 21.5, 22.5, 23.5, 24.5, 26.5],
    "Tiros OT": [10.5, 11.5, 12.5, 13.5, 14.5, 15.5],
    "Tarjetas": [2.5, 3.5, 4.5, 5.5]
}

resultados_probabilidades = {}

for categoria, umbrales in categorias_y_umbrales.items():
    resultados_probabilidades[categoria] = calcular_probabilidades_normal(df, "WOLVES", "MAN UNITED", categoria, umbrales)

resultados_probabilidades

{'Goles': {'Over/Under 0.5': {'Over': 0.9001692808321908,
   'Under': 0.09983071916780911},
  'Over/Under 1.5': {'Over': 0.7747019498197063, 'Under': 0.22529805018029375},
  'Over/Under 2.5': {'Over': 0.5895262487067776, 'Under': 0.4104737512932224},
  'Over/Under 3.5': {'Over': 0.3814142180206631, 'Under': 0.6185857819793369}},
 'Corners': {'Over/Under 7.5': {'Over': 0.80080250959953,
   'Under': 0.19919749040046997},
  'Over/Under 8.5': {'Over': 0.7093072348072346, 'Under': 0.29069276519276543},
  'Over/Under 9.5': {'Over': 0.6018863947269747, 'Under': 0.3981136052730253},
  'Over/Under 10.5': {'Over': 0.48608120336355787,
   'Under': 0.5139187966364421},
  'Over/Under 11.5': {'Over': 0.3714461888171522,
   'Under': 0.6285538111828478}},
 'Tiros': {'Over/Under 20.5': {'Over': 0.8204567090763151,
   'Under': 0.17954329092368487},
  'Over/Under 21.5': {'Over': 0.7814323362956639,
   'Under': 0.21856766370433617},
  'Over/Under 22.5': {'Over': 0.7379286174960807,
   'Under': 0.262071382

In [23]:
# Calcular probabilidades Over/Under para las demás categorías

# Corners
resultados_corners = calcular_probabilidades(df, "WOLVES", "MAN UNITED", "Corners", [7.5, 8.5, 9.5, 10.5, 11.5])

# Tiros
resultados_tiros = calcular_probabilidades(df, "WOLVES", "MAN UNITED", "Tiros", [20.5, 21.5, 22.5, 23.5, 24.5, 26.5])

# Tiros OT
resultados_tiros_ot = calcular_probabilidades(df, "WOLVES", "MAN UNITED", "Tiros OT", [10.5, 11.5, 12.5, 13.5, 14.5, 15.5])

# Tarjetas
resultados_tarjetas = calcular_probabilidades(df, "WOLVES", "MAN UNITED", "Tarjetas", [2.5, 3.5, 4.5, 5.5])

(resultados_corners, resultados_tiros, resultados_tiros_ot, resultados_tarjetas)

({7.5: {'Over': 0.8643311851228991, 'Under': 0.13566881487710092},
  8.5: {'Over': 0.7784141715202053, 'Under': 0.2215858284797947},
  9.5: {'Over': 0.6722680197941258, 'Under': 0.3277319802058742},
  10.5: {'Over': 0.5542436082320328, 'Under': 0.4457563917679673},
  11.5: {'Over': 0.43494187619848834, 'Under': 0.5650581238015117}},
 {20.5: {'Over': 0.9557035084556622, 'Under': 0.044296491544337846},
  21.5: {'Over': 0.9328170651886445, 'Under': 0.06718293481135545},
  22.5: {'Over': 0.9022398853043012, 'Under': 0.09776011469569873},
  23.5: {'Over': 0.8631637687750862, 'Under': 0.13683623122491387},
  24.5: {'Over': 0.8153071558233834, 'Under': 0.18469284417661658},
  26.5: {'Over': 0.6954333836406931, 'Under': 0.30456661635930693}},
 {10.5: {'Over': 0.37978084725573336, 'Under': 0.6202191527442666},
  11.5: {'Over': 0.2699269870292804, 'Under': 0.7300730129707196},
  12.5: {'Over': 0.18110665361205913, 'Under': 0.8188933463879409},
  13.5: {'Over': 0.11481675275579961, 'Under': 0.885

In [41]:
equipo_local_data

,Equipo,Goles L,Goles V,Goles L EC,Goles V EC,Tiros L,Tiros V,Tiros L EC,Tiros V EC,Tiros OT L,...,Tiros OT L EC,Tiros OT V EC,Tarjetas L,Tarjetas V,Tarjetas L EC,Tarjetas V EC,Corners L,Corners V,Corners L EC,Corners V EC
0,WOLVES,1.428571,2.0,1.142857,1.4,12.714286,11.8,12.571429,13.6,4.857143,...,3.714286,4.4,1.857143,1.8,3.285714,1.6,5.714286,2.8,4.857143,8.4
